<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/exercices-solutions/04_Transfer_Learning_with_TensorFlow_Part_1_Feature_Extraction_Exercice_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04 - Transfer Learning with TensorFlow Part 1: Feature Extraction Exercice Solutions

1. Build and fit a model using the same data we have here but with the MobileNetV2 architecture feature extraction from TensorFlow Hub. How does it perform compared to our other mdoels?
2. Name 3 different image classification models on TensorFlow Hub that we haven't used.
3. Build a model to classify images of two different things you've taken photos of.
- You can use any feature extraction layer from TensorFlow Hub you like for this.
- You should aim to have at least 10 images of each class, for example to build a fridge versus oven classifier, you'll want 10 images of fridges and 10 images of ovens.
4. What is the current best performing model on ImageNet?